In [11]:
import subprocess
import json
import csv
import time

seconds_hour = 3600
ais_catcher_command = ["AIS-catcher", "-T", "3600", "-o", "5"]

process = subprocess.Popen(ais_catcher_command, stdout=subprocess.PIPE, universal_newlines=True)
start_time = time.time()
with open("ais_messages.csv", "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in process.stdout:
        data = json.loads(line)
        print(line)
        try:
            csv_writer.writerow([data["mmsi"], data["lat"], data["lon"], data["heading"], data["course"]])
        except:
            pass
        if time.time() - start_time >= seconds_hour:
            break
process.wait()


{"class":"AIS","device":"AIS-catcher","scaled":true,"channel":"B","nmea":["!AIVDM,1,1,,B,403Ot`QvMQ?N9o@7:jK?fhO025qp,0*71"],"type":4,"repeat":0,"mmsi":3669154,"timestamp":"2023-06-02T15:30:09Z","year":2023,"month":6,"day":2,"hour":15,"minute":30,"second":9,"accuracy":true,"lon":-122.309402,"lat":47.615467,"epfd":15,"epfd_text":"Undefined","raim":true,"radio":24184}

{"class":"AIS","device":"AIS-catcher","scaled":true,"channel":"B","nmea":["!AIVDM,1,1,,B,15NeuhPP?w<tSF0l4Q@>4?wp0d3l,0*42"],"type":1,"repeat":0,"mmsi":367754690,"status":0,"status_text":"Under way using engine","turn_unscaled":-128,"turn":-128,"accuracy":false,"lon":181.000000,"lat":91.000000,"course":360.000000,"heading":511,"second":60,"maneuver":0,"raim":false,"radio":180468}

{"class":"AIS","device":"AIS-catcher","scaled":true,"channel":"B","nmea":["!AIVDM,1,1,,B,B5NgT6P005l0KsVl=WkQ3wU5kP06,0*6B"],"type":18,"repeat":0,"mmsi":367780890,"speed":0.000000,"accuracy":false,"lon":-122.327904,"lat":47.646072,"course":360.00

0

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap


df = pd.read_csv('ais_messages.csv', sep=',')

grouped_data = df.groupby('MMSI')
grouped_data = sorted(grouped_data, key=lambda x: len(x[1]), reverse=True)  

for name, group in grouped_data:
    # Create a map centered around the first latitude and longitude of the trajectory
    ship_map = folium.Map(location=[group['latitude'].iloc[0], group['longitude'].iloc[0]], zoom_start=10)

    trajectory = group[['latitude', 'longitude']].values.tolist()

    folium.PolyLine(trajectory, color='blue', weight=12.5, opacity=1).add_to(ship_map)

    img_file = f'maps/trajectory_{name}.html'
    ship_map.save(img_file)